In [1]:
import swmmAPI_v2 as sm
from pyswmm import Simulation, Links, Nodes
import numpy as np

In [2]:
swmmINP = 'YOUR_SWMM_INPUT_FILE_HERE.inp'
outF = 'OUTPUT_FILE_NAME.out'
metaCSV = 'YOUR_META_TABLE_HERE.csv'


offset = -479.755 # To Detroit datum
control = True
derivative = False
control_step = 0 # seconds between actions 

In [3]:
model = sm.swmmINP(swmmINP)
model.set_dicts()
model.set_geo_dicts()

In [4]:
ControlPoints = sm.make_control_points('input_files/ControlPoints.csv')
DownstreamPoints = sm.make_downstream_points('input_files/DownstreamPoints.csv')
for c in ControlPoints:
    c.get_model_info(model)
    
    # Correct Max Depth and Volume to represent that of the real system.
    if c.u_name == '2909':
        c.max_depth = 22
        c.max_vol = 4224919.6
        
for d in DownstreamPoints:
    d.get_model_info(model)
    if d.d_name == '2909':
        d.max_depth = 22
        d.max_vol = 4224919.6

Downstream Points
2909 storage
Downstream Points
1715 link
Downstream Points
1686 link
Downstream Points
1503 link
Downstream Points
20100 link
Downstream Points
21450 link
Downstream Points
27450 link


In [5]:
for c in ControlPoints:
    print(c.u_name,c.max_vol,c.measure,c.location, c.u_param)
    
for d in DownstreamPoints:    
    print(d.d_name, d.max_vol, d.epsilon, d.set_point)

17311 307544.99999999994 volume 17311 -- CC Forebay 0.684
5220 603200.0 volume 5220 -- Freud Storm PS 0.029
5010 400000.0 volume 5010 -- CC PS to RTB 0.097
5010 400000.0 volume 5010 -- CC PS to DRI 0.651
2909 4224919.6 volume 2909 -- CC RTB Dewatering 0.419
2912 728360.5 volume 2912 --CC Forebay Reguator Gates 1.0
1700 34925.5368 volume 1700 -- Fairview PS 0.459
1509 652612.83 volume ISD002 1.0
RC1954 11654400.0 volume ISD003 1.0
1520 350055.10301889625 volume ISD004 1.0
2020 170979.216676216 volume ISD005 1.0
2150 881379.9058003828 volume ISD006 1.0
2180 142839.77492847136 volume ISD007 1.0
2190 203409.87763922344 volume ISD008 1.0
2198 262659.13698868186 volume ISD009 1.0
2220 216035.20220322264 volume ISD010 1.0
2750 303793.9730975415 volume ISD011 1.0
2525 365410.4225114798 volume ISD012 1.0
2360 299246.3213819294 volume ISD013 1.0
2909 4224919.6 0.588 0.062
1715 163114.63216703566 0.721 0.802
1686 170679.58727687987 0.373 0.921
1503 331623.19999999995 2.0 0.3
20100 76969.020012949

In [9]:
%%time
with Simulation(swmmINP,outputfile = outF) as sim:
    run = sm.system(sim,offset = offset, control = control, control_step = control_step) # Offset = 0, by default
    run.timestep = model.options['ROUTING_STEP']
    nodes = Nodes(sim)
    links = Links(sim)
    
    run.groups = max(d.group for d in DownstreamPoints)
    
    for c in ControlPoints:
        c.set_vars(nodes,links)
    for d in DownstreamPoints:
        d.set_vars(nodes,links)
    
    
    # DO ONCE

    n = len(ControlPoints)

    # make group matrix. dimensions of matrix are [ # of groups , # of Control Elements ]
    # Put 1 in row if Control Element is part of group
    groupM = np.zeros((run.groups,n))
    for c,i in zip(ControlPoints,range(0,n)):
        groupM[c.group-1,i] = 1

    # Make arrays for price calculations
    uparam = np.array([c.u_param for c in ControlPoints])
#     dsparam = np.array(c.ds_param for c in ControlPoints)
    dparam = np.array([d.epsilon for d in DownstreamPoints])
    setpts = np.array([d.set_point for d in DownstreamPoints])
    n_tanks_1 = np.sum(groupM,axis=1)+1 # Do once
    q_goal = [c.q_goal for c in ControlPoints]
    max_vol = [d.max_vol for d in DownstreamPoints]
    T = max(run.timestep,run.control_step) # Returns the value used to calc the Qavailable
    
    Vol_up = np.array([0 for c in ControlPoints])
    Vol_down = np.array([0 for d in DownstreamPoints])
    
    run.last_action = sim.start_time                     
    print('Running Simulation...')
    for step in sim:
        take_action = False
        b = sim.current_time - run.last_action
        if b.total_seconds() >= run.control_step:
            take_action = True
        
        if run.control and take_action:
            run.last_action = sim.current_time
            
            # Grab y(t-1) for derivatives... do later
            last_up = Vol_up
            Last_down = Vol_down

            # Normalize Upstream and Downstream Volumes 
            Vol_up = np.array([c.u_var.volume / c.max_vol])
            Vol_down = np.array([d.d_var.volume / d.max_vol])
            
            # Wealth of Each Upstream Element
            Pwealth = uparam * Vol_up # Add Derivative Pricing Here too
            # Group Wealth, sum of Pwealth_i for each group
            Gwealth = np.mat(Pwealth)*np.mat(groupM).transpose()
            
            # Downstream Cost... need better name
            Dcost = ( Vol_down - setpts ) * dparam
            
            # Calc Pareto Price for each Group, give to each upstream element
            Pareto = ( Gwealth + Dcost ) / n_tanks_1
            Pareto_mat = np.mat(Pareto) * np.mat(groupM)
            
            # Calculate Purchasing Power of Each Upstream Point
            Ppower = Pwealth - Pareto_mat
            Ppower[Ppower < 0] = 0
            
            # Calculate Available Volume Downstream
            f = (1 - Vol_down) * max_vol
            Qavailable = f / T

            Qavailable_mat = np.mat(Qavailable) * np.mat(groupM)

            q_goal = np.array(Qavailable_mat) * np.array(Ppower)

            # Assign and send
            for q,c in zip(np.ndenumerate(q_goal),ControlPoints):
                c.q_goal = q[1] # q is a tuple with [0] index of array and [1] the value
                c.get_target_setting(run,nodes,links)
                
            run.actions.append([c.action for c in ControlPoints])
                
        else:
            for c in ControlPoints:
                c.check_flooding(run,nodes,links)

run.flood_count = sum(c.flood_count for c in ControlPoints)
print('Flood Count: ' , run.flood_count)
print("Saving...")

push_str = sm.push_meta(run,outF,ControlPoints,DownstreamPoints,derivative)
with open(metaCSV,'a+',newline='') as f:
    f.write(push_str)

print("Done.")

{'offset': -479.755, 'control': True, 'control_step': 0}
storage
link
link
link
link
link
link
Running Simulation...
Flood Count:  11291.0
Saving...
2
3
4
5
6
7
8
Done.
Wall time: 1h 7min 5s
